In [ ]:
#kafka_producer.py
from kafka import KafkaProducer
import csv
import json

def read_csv(file_path):
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            yield row

def main():
    producer = KafkaProducer(
        bootstrap_servers='localhost:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
    for data in read_csv('sample_data.csv'):
        producer.send('my_topic', value=data)
        
    producer.flush()
    producer.close()

if __name__ == '__main__':
    main()


In [ ]:
#kafka_consumer.py
from kafka import KafkaConsumer
import json
import mysql.connector

def insert_into_db(data):
    conn = mysql.connector.connect(
        host='localhost',
        user='your_username',
        password='your_password',
        database='your_database'
    )
    cursor = conn.cursor()
    
    add_data = ("INSERT INTO your_table "
                "(id, name, age, city) "
                "VALUES (%s, %s, %s, %s)")
    
    data_tuple = (data['id'], data['name'], data['age'], data['city'])
    cursor.execute(add_data, data_tuple)
    
    conn.commit()
    cursor.close()
    conn.close()

def main():
    consumer = KafkaConsumer(
        'my_topic',
        bootstrap_servers='localhost:9092',
        value_deserializer=lambda m: json.loads(m.decode('utf-8'))
    )
    
    for message in consumer:
        data = message.value
        insert_into_db(data)

if __name__ == '__main__':
    main()


: 